# Demo: Phân Bổ Ngược Forecast từ Cohort xuống Loan Level

Notebook này demo cách sử dụng module `allocation.py` để:
1. Phân bổ kết quả forecast từ cohort-level xuống loan-level
2. Validate tổng EAD có khớp không
3. Enrich thêm thông tin customer/product

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path(".").resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np

from src.config import CFG, BUCKETS_CANON
from src.data_loader import load_data
from src.rollrate.allocation import (
    allocate_forecast_to_loans,
    allocate_forecast_to_loans_simple,
    validate_allocation,
    enrich_loan_forecast,
)

print("✅ Import thành công")

## 1. Load Data

In [ ]:
# Load raw loan-level data
df_raw = load_data("path/to/your/parquet")

# Load lifecycle forecast (giả sử bạn đã có từ bước trước)
# df_lifecycle_final = pd.read_excel("outputs/Lifecycle_Final.xlsx", sheet_name="CDLPIL_DEL90")
# Hoặc từ memory nếu bạn vừa chạy xong calibration

print(f"📦 Loaded {len(df_raw):,} loan-level rows")
print(f"📦 Lifecycle forecast: {len(df_lifecycle_final):,} cohort-level rows")

## 2. Phân Bổ Ngược - Phương Pháp 1: Proportional

Mỗi loan nhận EAD theo tỷ lệ EAD hiện tại trong cohort.

## 2A. Ví Dụ 1: IFRS9 12-Month ECL (Stage 1)

Tính ECL cho 12 tháng tới.

In [ ]:
# Xác định MOB hiện tại
latest_cutoff = df_raw["CUTOFF_DATE"].max()
df_current = df_raw[df_raw["CUTOFF_DATE"] == latest_cutoff]
current_mob_max = df_current["MOB"].max()

print(f"📍 MOB hiện tại (max): {current_mob_max}")
print(f"📍 Target MOB (12 tháng tới): {current_mob_max + 12}")

# Phân bổ tại MOB = current + 12
df_allocated_12m = allocate_forecast_to_loans_simple(
    df_lifecycle_final=df_lifecycle_final,
    df_raw=df_raw,
    target_mob=current_mob_max + 12,  # 12 tháng tới
    forecast_only=True,
)

print(f"\n✅ Kết quả: {len(df_allocated_12m):,} loans với forecast tại MOB {current_mob_max + 12}")
df_allocated_12m.head()

## 2B. Ví Dụ 2: IFRS9 Lifetime ECL (Stage 2/3)

Tính ECL cho toàn bộ vòng đời (đến maturity).

In [ ]:
# Giả sử loan có term 36 tháng
maturity_mob = 36

print(f"📍 Target MOB (maturity): {maturity_mob}")

# Phân bổ tại MOB = maturity
df_allocated_lifetime = allocate_forecast_to_loans_simple(
    df_lifecycle_final=df_lifecycle_final,
    df_raw=df_raw,
    target_mob=maturity_mob,  # Maturity
    forecast_only=True,
)

print(f"\n✅ Kết quả: {len(df_allocated_lifetime):,} loans với forecast tại MOB {maturity_mob}")
df_allocated_lifetime.head()

## 2C. Ví Dụ 3: Tất Cả MOB (Portfolio Monitoring)

Lấy forecast cho tất cả MOB để vẽ curve.

In [ ]:
# Lấy tất cả MOB forecast
df_allocated_all = allocate_forecast_to_loans_simple(
    df_lifecycle_final=df_lifecycle_final,
    df_raw=df_raw,
    target_mob=None,  # Tất cả MOB
    forecast_only=True,
)

print(f"\n✅ Kết quả: {len(df_allocated_all):,} loan-level forecasts")
print(f"   MOB range: {df_allocated_all['TARGET_MOB'].min()} - {df_allocated_all['TARGET_MOB'].max()}")

# Xem phân phối theo MOB
print("\n📊 Số loans theo TARGET_MOB:")
print(df_allocated_all["TARGET_MOB"].value_counts().sort_index())

df_allocated_all.head()

## 2D. So Sánh: Proportional vs Simple

## 4. Validation: Kiểm Tra Tổng EAD

In [ ]:
# Validate phương pháp 1
compare_df = validate_allocation(
    df_allocated=df_allocated,
    df_lifecycle_final=df_lifecycle_final,
    group_cols=["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"]
)

# Xem các cohort có lỗi
errors = compare_df[compare_df["STATUS"] != "OK"]
if not errors.empty:
    print(f"\n⚠️ Có {len(errors)} cohorts có chênh lệch:")
    print(errors[["PRODUCT_TYPE", "VINTAGE_DATE", "MOB", "DIFF_PCT", "STATUS"]].head(20))
else:
    print("\n✅ Tất cả cohorts đều khớp (chênh lệch < 0.1%)")

## 5. Enrich: Thêm Thông Tin Customer/Product

In [ ]:
# Thêm các cột bổ sung từ df_raw
additional_cols = [
    "CUSTOMER_ID",
    "CUSTOMER_NAME",
    "BRANCH_CODE",
    "PRODUCT_NAME",
    "LOAN_TERM",
    "INTEREST_RATE",
    # ... các cột khác bạn cần
]

df_enriched = enrich_loan_forecast(
    df_allocated=df_allocated_simple,
    df_raw=df_raw,
    additional_cols=additional_cols,
)

print(f"\n✅ Enriched: {len(df_enriched.columns)} columns")
df_enriched.head()

## 6. Phân Tích Kết Quả

In [ ]:
# 6.1. Tổng EAD theo state forecast
print("📊 Tổng EAD theo STATE_FORECAST:")
state_summary = (
    df_enriched.groupby("STATE_FORECAST")["EAD_FORECAST"]
    .sum()
    .sort_values(ascending=False)
)
print(state_summary)

# 6.2. Tổng EAD theo product
print("\n📊 Tổng EAD theo PRODUCT_TYPE:")
product_summary = (
    df_enriched.groupby("PRODUCT_TYPE")["EAD_FORECAST"]
    .sum()
    .sort_values(ascending=False)
)
print(product_summary)

# 6.3. Số lượng loan theo state
print("\n📊 Số lượng loan theo STATE_FORECAST:")
loan_count = df_enriched["STATE_FORECAST"].value_counts()
print(loan_count)

## 7. Export Kết Quả

In [ ]:
# Export loan-level forecast
output_file = "outputs/Loan_Level_Forecast.xlsx"

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    # Sheet 1: Loan-level forecast
    df_enriched.to_excel(writer, sheet_name="Loan_Forecast", index=False)
    
    # Sheet 2: Summary by state
    state_summary.to_excel(writer, sheet_name="Summary_State")
    
    # Sheet 3: Summary by product
    product_summary.to_excel(writer, sheet_name="Summary_Product")
    
    # Sheet 4: Validation
    compare_df.to_excel(writer, sheet_name="Validation", index=False)

print(f"\n✅ Exported to {output_file}")

## 8. Use Case: Lọc Loan Theo Tiêu Chí

In [ ]:
# Ví dụ: Lọc các loan dự báo sẽ rơi vào DPD90+ tại MOB 12
high_risk_loans = df_enriched[
    (df_enriched["MOB"] == 12) &
    (df_enriched["STATE_FORECAST"].isin(["DPD90+", "DPD120+", "DPD180+", "WRITEOFF"]))
].copy()

print(f"\n⚠️ Có {len(high_risk_loans):,} loans dự báo rơi vào DPD90+ tại MOB 12")
print(f"   Tổng EAD: {high_risk_loans['EAD_FORECAST'].sum():,.0f}")

# Top 10 loans có EAD cao nhất
print("\n📋 Top 10 loans có EAD cao nhất:")
top_loans = high_risk_loans.nlargest(10, "EAD_FORECAST")
print(top_loans[["AGREEMENT_ID", "CUSTOMER_NAME", "STATE_FORECAST", "EAD_FORECAST"]])

## 9. Visualize

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 9.1. EAD distribution by state
plt.figure(figsize=(12, 6))
state_summary.plot(kind="bar")
plt.title("EAD Distribution by Forecast State")
plt.xlabel("State")
plt.ylabel("Total EAD")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 9.2. Loan count by state
plt.figure(figsize=(12, 6))
loan_count.plot(kind="bar", color="steelblue")
plt.title("Loan Count by Forecast State")
plt.xlabel("State")
plt.ylabel("Number of Loans")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()